In [15]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import csv
import math

from datetime import datetime
from sklearn.preprocessing import StandardScaler

In [2]:
!mv dataset/train.csv dataset/data.csv

mv: dataset/train.csv: No such file or directory


In [3]:
data_file_name = 'data.csv'
no_samples = 55423855

train_file_name = 'train_data.csv'
val_file_name = 'val_data.csv'
test_file_name = 'test_data.csv'

transformed_train_file_name = 'transformed_train_data.csv'
transformed_val_file_name = 'transformed_val_data.csv'
transformed_test_file_name = 'transformed_test_data.csv'

data_field_names = ['key', 'year', 'month', 'day', 'hour', 'minute', 'second', 
                    'passenger_count', 'distance', 'fare_amount']

target_column = 'fare_amount'

In [4]:
def get_source_file_path(file_name):
    return os.path.join(os.getcwd(), 'dataset', file_name)

In [5]:
data_file_path = get_source_file_path(data_file_name)

train_file_path = get_source_file_path(train_file_name)
val_file_path = get_source_file_path(val_file_name)
test_file_path = get_source_file_path(test_file_name)

In [6]:
def calculate_distance(pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude):
    if pickup_longitude == 0 or pickup_latitude == 0 or dropoff_latitude == 0 or dropoff_longitude == 0:
        return 0

    lat = abs(math.radians(pickup_latitude) - math.radians(dropoff_latitude))
    long = abs(math.radians(pickup_longitude) - math.radians(dropoff_longitude))

    a_lat = math.sin(lat / 2) * math.sin(lat / 2)
    c_lat = 2 * math.atan2(math.sqrt(a_lat), math.sqrt(1 - a_lat))
    lat_dist = 6371 * c_lat

    a_long = math.sin(long / 2) * math.sin(long / 2)
    c_long = 2 * math.atan2(math.sqrt(a_long), math.sqrt(1 - a_long))
    long_dist = 6371 * c_long

    return round(abs(lat_dist) + abs(long_dist), 1)

In [7]:
def split_source_file(file_columns, split_dist=(0.6, 0.1, 0.3)):
    if os.path.exists(train_file_path) and os.path.exists(val_file_path) and os.path.exists(test_file_path):
        return train_file_path, val_file_path, test_file_path

    setup_data_source(train_file_path, file_columns)
    setup_data_source(val_file_path, file_columns)
    setup_data_source(test_file_path, file_columns)

    train_percentage = split_dist[0]
    val_percentage = split_dist[1]
    test_percentage = split_dist[2]

    train_threshold = int(no_samples * train_percentage)
    val_threshold = train_threshold + int(no_samples * val_percentage)
    test_threshold = val_threshold + int(no_samples * test_percentage)

    train_file, train_writer = open_file_append(train_file_path, file_columns)
    val_file, val_writer = open_file_append(val_file_path, file_columns)
    test_file, test_writer = open_file_append(test_file_path, file_columns)

    with open(data_file_path) as data_file:
        data_file_reader = csv.DictReader(data_file)

        next(data_file_reader, None)
        for index, entry in enumerate(data_file_reader):
            if index < train_threshold:
                write_entry(entry, train_writer)
            if train_threshold < index < val_threshold:
                write_entry(entry, val_writer)
            if index >= val_threshold:
                write_entry(entry, test_writer)

    train_file.close()
    val_file.close()
    test_file.close()

def setup_data_source(file_path, file_columns):
    with open(file_path, 'w') as file:
        train_writer = csv.DictWriter(file, fieldnames=file_columns)
        train_writer.writeheader()

def open_file_append(file_path, file_columns):
    file = open(file_path, 'a')
    writer = csv.DictWriter(file, fieldnames=file_columns)
    return file, writer

def write_entry(entry, writer):
    try:
        fare_amount = float(entry['fare_amount'])
        pickup_longitude = float(entry['pickup_longitude']) if entry['pickup_longitude'] != '' else 0
        pickup_latitude = float(entry['pickup_latitude']) if entry['pickup_latitude'] != '' else 0
        dropoff_longitude = float(entry['dropoff_longitude']) if entry['dropoff_longitude'] != '' else 0
        dropoff_latitude = float(entry['dropoff_latitude']) if entry['dropoff_latitude'] != '' else 0
        passenger_count = int(entry['passenger_count'])
        distance = calculate_distance(pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude)

        date = datetime.strptime(str(entry['pickup_datetime']), '%Y-%m-%d %H:%M:%S UTC')

        writer.writerow({'year': date.year, 'month': date.month, 'day': date.day, 'hour': date.hour,
                         'minute': date.minute, 'second': date.second, 'passenger_count': passenger_count,
                         'distance': distance, 'fare_amount': fare_amount})
    except ValueError:
        print('entry has invalid values')

In [8]:
!head {train_file_path}

In [9]:
!head {val_file_path}

In [10]:
!head {test_file_path}

In [16]:
def scale_dataset_save_to_disk(file_path, scaled_file_name):
    scaled_file_path = get_source_file_path(scaled_file_name)
    
    if os.path.exists(scaled_file_path):
        return scaled_file_path
    
    df = pd.read_csv(file_path)
    features = df[['year', 'month', 'day', 'hour', 'minute', 'second', 'passenger_count', 'distance']]
    target = df[['fare_amount']]
    
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features.values)
    features = pd.DataFrame(data=scaled_features, columns=['year', 'month', 'day', 'hour', 'minute', 'second', 
                                                           'passenger_count', 'distance'])
    
    df = pd.concat([features, target], axis=1)
    
    df.to_csv(scaled_file_path)
    
    return scaled_file_path

In [17]:
transformed_train_file_path = scale_dataset_save_to_disk(train_file_path, transformed_train_file_name)

In [18]:
transformed_val_file_path = scale_dataset_save_to_disk(val_file_path, transformed_val_file_name)

In [19]:
transformed_test_file_path = scale_dataset_save_to_disk(test_file_path, transformed_test_file_name)

In [20]:
!head {transformed_train_file_path}

,year,month,day,hour,minute,second,passenger_count,distance,fare_amount
0,-0.9350459117422577,-1.5328556698155498,-1.2338102099347008,0.38204534784905536,1.2936431424362251,0.009233325222440213,-0.5174363865786075,0.04446917833210273,16.9
1,-0.39748218463311386,0.5045985327022482,0.26312360219004244,-2.0730194171825924,0.3127240921709917,-0.8142840494483844,0.2374902859442906,-0.03609809444894267,5.7
2,0.14008154247602994,-0.6596610115936363,0.6085698665265217,-1.4592532259246807,0.02421848915180535,1.3474490590625303,-0.5174363865786075,-0.026332364414876568,7.7
3,-0.9350459117422577,-0.9507258976676074,-0.7732151908193953,-0.9989285824812466,1.235942021832388,-0.8142840494483844,-0.5174363865786075,-0.028773796923393095,5.3
4,-0.39748218463311386,-1.5328556698155498,-1.1186614551558745,-0.6920454868522906,1.1782409012285506,1.50185856681331,-0.5174363865786075,-0.003545661002055645,12.1
5,0.14008154247602994,1.3777931909241616,0.49342111174769526,0.9958115391069673,0.3127240921709917

In [21]:
!head {transformed_val_file_path}

,year,month,day,hour,minute,second,passenger_count,distance,fare_amount
0,-0.3972282839191653,-0.659661598670524,-1.3490861966311478,-0.2309646690446781,1.3511742418955914,1.45104470393324,-0.5122455125558507,-0.03572869945199469,6.1
1,0.14054723917577983,1.0868256634293523,0.032602915604914594,0.9967026829662163,-0.7837596882284843,2.068767061227297,-0.5122455125558507,0.01562827620813575,14.0
2,0.678322762270725,-0.36858038832054457,1.2991512684879718,-1.9190072780596579,-0.26445143495506046,-0.35064550484109264,-0.5122455125558507,-0.019679644558203927,14.0
3,-0.9350038070141105,-0.07749917797056521,1.0688697497819615,-1.151715183052849,0.4279595694095046,-0.8139372728116353,-0.5122455125558507,-0.0036305896644131656,8.9
4,0.14054723917577983,-1.2418240193704826,-1.4642269559841532,0.0759521689580455,-1.5338716096234297,-0.8139372728116353,-0.5122455125558507,-0.04214832140951099,7.7
5,0.678322762270725,0.21358203237941414,0.14774367495791982,1.3036195209689398,0.7741650715917872,-0

In [22]:
!head {transformed_test_file_path}

,year,month,day,hour,minute,second,passenger_count,distance,fare_amount
0,-0.9351129252630839,-0.9507243643088166,1.529731604596719,0.3819225803114648,-1.7068317060747074,-0.8138993056384554,-0.5151710215978829,-0.03301486896884507,7.3
1,0.14020118499052908,0.21365098510856032,-0.7732763350256169,-0.6920030480742525,-0.49500033437196256,1.1426456764789068,-0.5151710215978829,0.004850477382570455,12.5
2,0.6778582401173355,-0.36853668960012814,-1.5793291138934344,0.6887584741359555,0.5437122699446758,-0.5564591764124867,1.7398248197548647,-0.005850598760220888,11.5
3,1.2155152952441421,-1.532912039017505,-1.233877922950084,-1.9193466233722152,-0.7835316133488065,-0.8138993056384554,2.4914901002057803,-0.050301222737969546,3.5
4,-0.9351129252630839,-0.36853668960012814,-0.3126747471011497,0.8421764210482008,0.02435596778635663,-0.29901904718651795,1.7398248197548647,-0.03795382718859492,6.1
5,-0.9351129252630839,-0.36853668960012814,1.2994308106344854,0.8421764210482008,-0.379587822781224

In [20]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, 
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  
  return dataset

def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [21]:
raw_data = get_dataset(data_file_path)
show_batch(raw_data)

